# Exercícios em Python 6: Biblioteca padrão python, expressões regulares, csv, xml



## Preparando o ambiente

Execute o código a seguir.

In [1]:
!git clone https://gitlab.uspdigital.usp.br/thiago/curso_ai_python.git
import sys
sys.path.append('curso_ai_python')

Cloning into 'curso_ai_python'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 77 (delta 20), reused 0 (delta 0), pack-reused 33
Receiving objects: 100% (77/77), 26.67 KiB | 223.00 KiB/s, done.
Resolving deltas: 100% (36/36), done.


Se o bloco acima foi executado corretamente, você deve ver o contéudo do diretório ```curso_ai_pytho`/dados```:

In [2]:
!ls -la curso_ai_python/dados/

total 32
drwxr-xr-x 2 root root  4096 May 24 01:13 .
drwxr-xr-x 5 root root  4096 May 24 01:13 ..
-rw-r--r-- 1 root root    20 May 24 01:13 dados01.txt
-rw-r--r-- 1 root root    50 May 24 01:13 dados02.txt
-rw-r--r-- 1 root root 12633 May 24 01:13 telefones_epusp.txt


## 6.1 Leitura de dados em CSV

O módulo ```csv``` em Python lê arquivos em formato CSV.

A função ```csv.DictReader(arquivo)``` retorna uma lista de dicionários, um para cada linha do arquivo.

Por exemplo, o código a seguir mostra o total de alunos estrangeiros no município de Araraquara.

In [12]:
import urllib.request
import csv
import io
import tempfile
import ssl

# Suprime verificação de SSL por problemas
#  com a ICP Brasil
ignorar_ssl = ssl.create_default_context()
ignorar_ssl.check_hostname = False
ignorar_ssl.verify_mode = ssl.CERT_NONE

recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3) # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso,encoding='utf-8'), delimiter=';')
total = 0
for entrada in leitorcsv:
  if entrada['MUN']=='ARARAQUARA':
    total += int(entrada[' Nº ALUNOS '])
print(total)

50


### Exercício 6.1.1

Encontre o total de alunos provenientes do Uruguai matriculados em escolas de SP.
O campo que contém o país é o campo com nome ```DS_PAIS```.
Atribua sua resposta à variável ```total_uruguai```.


In [24]:
recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3) # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso,encoding='utf-8'), delimiter=';')

total_uruguai = 0
for entrada in leitorcsv:
  if entrada['DS_PAIS']=='Uruguai':
    total_uruguai += int(entrada[' Nº ALUNOS '])
print(total_uruguai)

18


A resposta deve ser 18

### Exercício 6.1.2

Quantos *distritos* distintos de escolas existem nesta lista? O campo que contém o tipo é ```DISTR```.
Qual o tipo com a maior quantidade de alunos?

Sugestão: Crie um dicionário.


In [49]:
recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3) # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso,encoding='utf-8'), delimiter=';')

total_uruguai = 0
dict_distritos = {}

for entrada in leitorcsv:
  dict_distritos[entrada['DISTR']] = 0

recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3) # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso,encoding='utf-8'), delimiter=';')

for entrada in leitorcsv:
  dict_distritos[entrada['DISTR']] += int(entrada[' Nº ALUNOS '])

In [62]:
lista_distritos = [(value, key) for key, value in dict_distritos.items()]

print(f'O maior distrito é {max(lista_distritos)[1]} com {max(lista_distritos)[0]} alunos')

O maior distrito é VILA MARIA com 642 alunos


A resposta deve ser 419 distritos distintos. O tipo com a maior quantidade é o ```VILA MARIA``` com 642 alunos.

## 6.2 Processamento de XML

A url ```https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarDia(dataCotacao=@dataCotacao)?@dataCotacao='04-23-2020'&$top=100&$format=xml&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao``` recupera o relatório de cotação do dólar do Banco central em formato xml e grava-o no arquivo cotacao_dolar.xml.



In [63]:
req=urllib.request.urlopen("https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2024'&@dataFinalCotacao='05-03-2024'&$top=200&$format=xml&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao")
ff = open("cotacao_dolar.xml", "wb")
ff.write(req.read())
ff.close()



O código abaixo usa xml para encontrar o valor da cotação no dia 17/03/2024

In [64]:
import xml.etree.ElementTree
raiz = xml.etree.ElementTree.parse("cotacao_dolar.xml").getroot()
for el in raiz.iter('{http://docs.oasis-open.org/odata/ns/metadata}properties'):
  hora = el.find('{http://docs.oasis-open.org/odata/ns/data}dataHoraCotacao')
  if hora.text.startswith('2024-05-02'):
    no_venda = el.find('{http://docs.oasis-open.org/odata/ns/data}cotacaoVenda')
    valor = float(no_venda.text)
    print(valor)



5.1184


Exercício:

Encontre o *menor* valor de cotação do dólar para venda (campo com etiqueta ```{http://docs.oasis-open.org/odata/ns/data}cotacaoVenda'```)

In [75]:
import xml.etree.ElementTree
raiz = xml.etree.ElementTree.parse("cotacao_dolar.xml").getroot()
list_valores = []
for el in raiz.iter('{http://docs.oasis-open.org/odata/ns/metadata}properties'):
  hora = el.find('{http://docs.oasis-open.org/odata/ns/data}dataHoraCotacao')
  no_venda = el.find('{http://docs.oasis-open.org/odata/ns/data}cotacaoVenda')
  list_valores.append(float(no_venda.text))

min(list_valores)

4.8543

A resposta deve ser 4.8543

## 6.3  Expressões Regulares

A linguagem Python tem uma biblioteca de expressões regulares.

A função ```re.search(exp, string)``` verifica se uma string é aderente a um determinado padrão.

### Exercícios:

O arquivo em ```/content/gdrive/My Drive/cursoai_python_aula_03/dados/telefones_epusp.txt``` contém uma lista telefônica em um texto não-estruturado com telefones úteis da escola politécnica.

Eis o conteúdo de suas 10 primeiras linhas:

In [76]:
arquivo = open("/content/curso_ai_python/dados/telefones_epusp.txt", "rt")
for i in range(10):
  print(arquivo.readline())
arquivo.close()

Diretoria

Profa. Dra. Liedi Legi Bariani Bernucci

Diretora 	3091 5221 / 5223 / 5550

Prof. Reinaldo Giudici

Vice-Diretor

Secretaria: Sueli Rossetti / Ângela de Jesus

Serviço Técnico de Planejamento e Gestão

Enaége Dalan Sant´Ana 	3091 5534

Seção Técnica de Qualidade e Processos

Patricia Sandes 	3091 5577



### 6.3.1 Conte o número de linhas com telefones

A expressão regular ```([0-9]+([\s-]+[0-9]+)*)(\s?/\s?([0-9]+([\s-]+[0-9]+)*))*$``` casa com um ou mais números de telefone ao final de uma linha.

Verifique usando o módulo re quantas linhas atendem este padrão.

In [102]:
import re

arquivo = open("/content/curso_ai_python/dados/telefones_epusp.txt", "rt")
regex = re.compile(r'([0-9]+([\s-]+[0-9]+)*)(\s?/\s?([0-9]+([\s-]+[0-9]+)*))*$')

list_re = []

for line in arquivo.readlines():
  if regex.search(line) is not None:
    list_re.append(regex.search(line))
arquivo.close()

print(len(list_re))

255


Sua resposta deve ser 255 linhas